# Using `xgcm.transform` to interpolate to isopycnal space

In [ ]:
%load_ext watermark

import numpy as np
import xarray as xr
import xgcm

import pop_tools

%watermark -iv

## Load data

Read an existing sample  dataset for the tropical pacific. Unfortunately, this particular subset does not have `SALT`, so for demonstration purposes we set `SALT=35`.

In [ ]:
# open sample data
filepath = pop_tools.DATASETS.fetch('Pac_POP0.1_JRA_IAF_1993-12-6-test.nc')
ds = xr.open_dataset(filepath)

# get DZU and DZT, needed for operations later on
filepath_g = pop_tools.DATASETS.fetch("Pac_grid_pbc_1301x305x62.tx01_62l.2013-07-13.nc")
ds_g = xr.open_dataset(filepath_g)
ds.update(ds_g[["DZU", "DZT"]])

# There is no salinity, so let's  create a fake SALT variable
ds["SALT"] = 35 * xr.ones_like(ds.TEMP)
ds.SALT.attrs["grid_loc"] = ds.TEMP.attrs["grid_loc"]
ds

## Construct xgcm-compatible dataset and  xgcm grid object

In [ ]:
metrics = {
    ("X",): ["DXU", "DXT"],  # X distances
    ("Y",): ["DYU", "DYT"],  # Y distances
    ("Z",): ["DZU", "DZT"],  # Z distances
    ("X", "Y"): ["UAREA", "TAREA"],
}

# here we get the xgcm compatible dataset
grid, xds = pop_tools.to_xgcm_grid_dataset(
    ds,
    periodic=False,
    metrics=metrics,
    boundary={"X": "extend", "Y": "extend", "Z": "extend"},
)

xds["rho"] = pop_tools.eos(xds.SALT, xds.TEMP, depth=xds.z_t * 1e-2)
xds

## Visualize surface density field

In [ ]:
(xds.rho - 1000).isel(z_t=0).plot(robust=True)

## Regrid to density space using `grid.transform`

See https://xgcm.readthedocs.io/en/latest/transform.html for more

In [ ]:
regridded = grid.transform(
    xds.TEMP,
    axis="Z",
    target=np.arange(25, 30, 0.5),
    target_data=xds.rho - 1000,
    method="linear",
)

## Visualize temperature on a single isopycnal surface

In [ ]:
regridded.isel(rho=5).plot(robust=True)

## Find depth of isopycnal surface

In [ ]:
regridded.coords["z_t"] = grid.transform(
    xds.z_t.broadcast_like(xds.rho),
    axis="Z",
    target=np.arange(24, 30, 0.5),
    target_data=xds.rho - 1000,
    method="linear",
)
regridded.z_t.isel(rho=5).plot(robust=True)